In [39]:
import pandas as pd
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
%pwd

'/home/alex/code/Korsas/final_project/mariajoi/flightdelay/flightdelay/utils'

In [40]:
%cd ../flightdelay/utils
from mytrans import MyTrans

[Errno 2] No such file or directory: '../flightdelay/utils'
/home/alex/code/Korsas/final_project/mariajoi/flightdelay/flightdelay/utils


/home/alex/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [41]:
df_test = pd.read_csv("../data/test_sample.csv")
df_test = df_test.drop("Unnamed: 0", axis = 1)


In [42]:
df_train = pd.read_csv("../data/train_sample.csv")
df_train = df_train.drop("Unnamed: 0", axis = 1)

In [43]:

X_test = df_test.drop(columns=['Tail_Number', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay',
             'Distance', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15',
       'ArrivalDelayGroups','CRSDepTime', 'CRSArrTime' , 'Cancelled', 'CancellationCode',
       'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'DepTime', 'DepDelay',
       'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups',
        'DestCityName', 'DestState', 'OriginCityName', 'OriginState', 'DestAirportID',
     'Flight_Number_Operating_Airline', 'OriginAirportID','FlightDate','DayofMonth'])

X_train = df_train.drop(columns=['Tail_Number', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay',
             'Distance', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15',
       'ArrivalDelayGroups','CRSDepTime', 'CRSArrTime' , 'Cancelled', 'CancellationCode',
       'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'DepTime', 'DepDelay',
       'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups',
        'DestCityName', 'DestState', 'OriginCityName', 'OriginState', 'DestAirportID',
     'Flight_Number_Operating_Airline', 'OriginAirportID','FlightDate','DayofMonth'])

In [44]:
df_train["BadFlight"] = ((df_train['ArrivalDelayGroups'] > 0) | (df_train['Cancelled'] > 0)).astype(int)
y_train = df_train["BadFlight"]

df_test["BadFlight"] = ((df_test['ArrivalDelayGroups'] > 0) | (df_test['Cancelled'] > 0)).astype(int)
y_test = df_test["BadFlight"]

In [45]:
trans = MyTrans()
trans.fit(X_train)
X_train_trans =trans.transform(X_train)

In [46]:
trans = MyTrans()
trans.fit(X_test)
X_test_trans =trans.transform(X_test)

In [47]:
preproc = pickle.load(open("../pickle/preproc_nottrans_notfit_.pkl","rb"))
preproc


ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fec02504fa0>)])

# Model TPOT vs1

## Exporting to a pickle file

In [49]:
import os
from tpot import TPOTRegressor
from tpot import TPOTClassifier

tpot_config = {'preprocessor': [MyTrans(), 'passthrough'],}

model_reg = TPOTRegressor(generations=3, population_size=60, cv=5,random_state=42, verbosity=2)
model_classy = TPOTClassifier(generations=3, population_size=60, cv=5,random_state=42, verbosity=2)
# with awareness of a preproc in the pipeline to skip analyse
model_classy_ref = TPOTClassifier(config_dict=tpot_config,generations=3, population_size=60, cv=5,random_state=42, verbosity=2)



In [13]:
# Add estimator

# Create and Fit Pipeline
model_pipeline = make_pipeline(preproc, model_classy)
model_pipeline
model_pipeline.fit(X_train_trans,y_train)

/home/alex/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


                                                                                
Generation 1 - Current best internal CV score: 0.6893355642036425
                                                                                
Generation 2 - Current best internal CV score: 0.6893355642036425
                                                                                
Generation 3 - Current best internal CV score: 0.6893355642036425
                                                             
Best pipeline: KNeighborsClassifier(OneHotEncoder(input_matrix, minimum_fraction=0.25, sparse=False, threshold=10), n_neighbors=64, p=1, weights=distance)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fec5d515300>)])),
                ('tpotclassifier',
                 TPOTClassifier(generations=3, population_size=60,
                                random_state=42, verbosity=2))])

In [10]:
# %cd ../pickle
# # Export Pipeline as pickle file

# with open("fitted_model_pipeline_vs2.pkl", "wb") as file:
#     pickle.dump(model_pipeline, file)

/Users/ronjaboehmer/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
/Users/ronjaboehmer/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


/Users/ronjaboehmer/code/mariajoi/flightdelay/flightdelay/pickle


In [54]:
#print(X_test.columns)
X_test.columns = X_test.columns.str.strip()
print(model_pipeline.score(X_test, y_test))
#model_classy.export('tpot_exported_pipeline.py')

0.5386111111111112


In [53]:
features_to_predict = X_test[:2]
features_to_predict
proba_predictions = model_pipeline.predict_proba(features_to_predict)
proba_predictions

array([[0.49939452, 0.50060548],
       [0.41221995, 0.58778005]])